In [11]:
!pip3 install djitellopy
!pip install timm
from djitellopy import tello

import cv2
import torch
import urllib.request
import time
import numpy as np
import random
import matplotlib.pyplot as plt

In [3]:
model_type = "DPT_Large"

midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in /home/santos/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /home/santos/.cache/torch/hub/intel-isl_MiDaS_master


In [4]:
lower = np.array([0, 0, 11])
upper = np.array([0, 0, 20])

In [5]:
drone = tello.Tello()
drone.connect()
print(drone.get_battery())
drone.streamon()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


89


In [6]:
while True:   
    img = drone.get_frame_read().frame
    num = random.randint(1,10000)
    cv2.imwrite(f"img/{num}.jpg", img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_batch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()

    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)
    depth_map = (depth_map*255).astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)
    
    M = depth_map.shape[0]//4
    N = depth_map.shape[1]//4
    tiles = [depth_map[x:x+M,y:y+N] for x in range(0,depth_map.shape[0],M) for y in range(0,depth_map.shape[1],N)]
    
    d1 = cv2.vconcat([tiles[0], tiles[1], tiles[2], tiles[3]])
    d2 = cv2.vconcat([tiles[4], tiles[5], tiles[6], tiles[7]])
    depth_map = cv2.hconcat([d1, d2])
    
    cv2.imwrite(f"img/depth_{num}.jpg", depth_map)
    output = cv2.imread(f"img/depth_{num}.jpg")
    
    output = cv2.cvtColor(output, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(output, lower, upper)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) != 0:
        for contour in contours:
            if cv2.contourArea(contour) > 500:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(mask, (x, y), (x + w, y + h), (0, 0, 255), 3)
                
    #cv2.imwrite(f"img/{num}(1).jpg", mask)
    
    cv2.waitKey(1)

[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] decode_slice_header error
[h264 @ 0x5611b39af4c0] no frame!
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] decode_slice_header error
[h264 @ 0x5611b39af4c0] no frame!
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] decode_slice_header error
[h264 @ 0x5611b39af4c0] no frame!
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] decode_slice_header error
[h264 @ 0x5611b39af4c0] no frame!
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] non-existing PPS 0 referenced
[h264 @ 0x5611b39af4c0] decode_slice_header error
[h264 @ 0x5611b39af4c0] no frame!
[h264 @ 0x5611b39af4c0] non-existing PPS

KeyboardInterrupt: 

In [9]:
path = "3047"
output = cv2.imread(f"img/{path}.jpg")
    
output = cv2.cvtColor(output, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(output, lower, upper)
    
cv2.imwrite(f"img/{path}({random.randint(1, 100)}).jpg", mask)

True

In [13]:
def concatenate(img):
    M = img.shape[0]//4
    N = img.shape[1]//4
    tiles = [img[x:x+M,y:y+N] for x in range(0,img.shape[0],M) for y in range(0,img.shape[1],N)]
    
    d1 = cv2.vconcat([tiles[0], tiles[1], tiles[2], tiles[3]])
    d2 = cv2.vconcat([tiles[4], tiles[5], tiles[6], tiles[7]])
    depth_map = cv2.hconcat([d1, d2])
    
    cv2.imwrite("img/test.jpg", depth_map)
    
i = cv2.imread(f"depth_3937.jpg")
concatenate(i)

[ WARN:0@2040.709] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('depth_3937.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'